# 1.构造net

## 1.1 构造Block
Block类是nn模块里提供的一个模型构造类，简单概括Block的基本功能：
+ 接收输入数据作为其前向传播函数的参数。
+ 通过使前向传播函数返回一个值来生成输出。请注意，输出的形状可能与输入的形状不同。
+ 计算其输出相对于其输入的梯度，可以通过其反向传播函数进行访问。通常，这是自动发生的。
+ 存储并提供对执行前向传播计算所需的那些参数。
+ 根据需要初始化模型参数。

我们可以继承它来定义我们想要的模型：
+ 通过重写Block类的__init__方法，构造模型结构
+ 通过重写Block类的forward方法，定义参数和正向传播过程。

In [1]:
from mxnet import npx, np,init
from mxnet.gluon import nn
npx.set_np()


X = np.random.uniform(size=(2, 20))

class MLP(nn.Block):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.hidden = nn.Dense(256, activation='relu')  # 定义一个256个隐藏单元的隐藏层
        self.out = nn.Dense(10) # 定义一个10纬的输出层
        
    # 这里的X为输入
    def forward(self, X):
        return self.out(self.hidden(X))

net = MLP()
net.initialize()
net(X)

array([[ 0.06240274, -0.03268593,  0.02582653,  0.02254181, -0.03728798,
        -0.04253785,  0.00540612, -0.01364185, -0.09915454, -0.02272737],
       [ 0.02816679, -0.03341204,  0.03565665,  0.02506384, -0.04136416,
        -0.04941844,  0.01738529,  0.01081963, -0.09932579, -0.01176296]])

## 1.2  构造Sequential
Block类是一个通用的部件。Sequential类继承自Block类。Sequential的作用是当模型的前向计算为简单串联各个层的计算时，可以通过更加简单的方式定义模型。它提供add函数来逐一添加串联的Block子类实例，而模型的forward就是将这些实例按添加的顺序逐一计算。

In [2]:
class MySequential(nn.Block):
    # 通过add方法将block 添加到_clildren这个字典里
    def add(self, block):
        self._children[block.name] = block
        
    # 使用_children中的每一个block处理X
    def forward(self, X):
        for block in self._children.values():
            X=block(X)
        return X

当MySequential调用我们的前向传播函数时，每个添加的块都按照添加的顺序执行。现在，我们可以使用我们的MySequential类来重新实现MLP 。

In [3]:
net = MySequential()
net.add(nn.Dense(256, activation='relu'))
net.add(nn.Dense(10))
net.initialize()
net(X)

array([[-0.03989595, -0.10414709,  0.06799038,  0.05245074,  0.0252606 ,
        -0.00640342,  0.04182098, -0.01665318, -0.02067345, -0.07863816],
       [-0.03612847, -0.07210435,  0.09159479,  0.07890773,  0.02494171,
        -0.01028665,  0.01732427, -0.02843244,  0.03772651, -0.06671703]])

## 1.3 在正向传播中执行代码
虽然Sequentail类使模型构造更加简单，需要定义forward函数，但直接继承Block类可以极大地拓展模型构造的灵活性。可以定义自己想要的数学运算，不局限于目前预设好的神经层。可以通过构造自己的block，在正向传播中进行一些计算。
构造一个稍微复杂点的网络FancyMLP。在这个网络中，我们通过get_constant函数创建训练中不被迭代的参数，即常数参数。在返回输出之前，我们的模型做了一些不寻常的事情。我们运行了一个while循环，测试它的条件 L1  规范大于  1 ，然后将输出向量除以  2 直到满足条件为止。最后，我们返回中的条目总和X。

In [5]:
class FixedHiddenMLP(nn.Block):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.rand_weight=self.params.get_constant('rand_weight', np.random.uniform(size=(20,20)))
        self.dense = nn.Dense(20, activation='relu')
        
    def forward(self, X):
        X=self.dense(X)
        # 使用创建的常数参数，以及NDArray的relu函数和dot函数
        X=npx.relu(np.dot(X, self.rand_weight.data())+1)
        # 复用全连接层。等价于两个全连接层共享参数
        X= self.dense(X)
        # 控制流
        while np.abs(X).sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net.initialize()
net(X)

## 1.4 Block混合使用
可以讲多个继承于Block的block混合使用。

In [6]:
class NestMLP(nn.Block):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.net = nn.Sequential()
        self.net.add(nn.Dense(64, activation='relu'),
                     nn.Dense(32, activation='relu'))
        self.dense = nn.Dense(16, activation='relu')

    def forward(self, X):
        return self.dense(self.net(X))

chimera = nn.Sequential()
chimera.add(NestMLP(), nn.Dense(20), FixedHiddenMLP())
chimera.initialize()
chimera(X)

array(0.8073602)

# 2.参数管理
## 2.1 访问模型参数
对于Sequential实例中含模型参数的层，我们可以通过Block类的params属性来访问该层包含的所有参数。访问多层感知机net中隐藏层的所有参数。索引0表示隐藏层为Sequential实例最先添加的层。

In [7]:
net = nn.Sequential()
net.add(nn.Dense(8, activation='relu'))
net.add(nn.Dense(1))
net.initialize()  # 初始化

X = np.random.uniform(size=(2, 4))
net(X)  

net[0].params, type(net[0].params)

(dense10_ (
   Parameter dense10_weight (shape=(8, 4), dtype=float32)
   Parameter dense10_bias (shape=(8,), dtype=float32)
 ),
 mxnet.gluon.parameter.ParameterDict)

+ 参数名称映射到参数实例的字典（类型为ParameterDict类）
+ 权重参数的名称为dense10_weight。它由net[0]的名称（dense10_）和自己的变量名（weight）组成
+ 该参数的形状为(256, 20)
+ 数据类型为32位浮点数（float32）

### 2.1.1 访问目标参数
访问的方法和访问字典差不多。下代码从第二个神经网络层提取偏差，该偏差返回一个参数类实例，并进一步访问该参数的值。

In [8]:
print(type(net[1].bias))
print(net[1].bias)
print(net[1].bias.data())

<class 'mxnet.gluon.parameter.Parameter'>
Parameter dense11_bias (shape=(1,), dtype=float32)
[0.]


In [9]:
net[1].weight.grad()

array([[0., 0., 0., 0., 0., 0., 0., 0.]])

### 2.1.2 访问所有参数

In [10]:
print(net[0].collect_params())
print(net.collect_params())

dense10_ (
  Parameter dense10_weight (shape=(8, 4), dtype=float32)
  Parameter dense10_bias (shape=(8,), dtype=float32)
)
sequential2_ (
  Parameter dense10_weight (shape=(8, 4), dtype=float32)
  Parameter dense10_bias (shape=(8,), dtype=float32)
  Parameter dense11_weight (shape=(1, 8), dtype=float32)
  Parameter dense11_bias (shape=(1,), dtype=float32)
)


## 2.2 初始化参数
默认情况下，MXNet通过从均匀分布中随机绘制来初始化权重参数 $ U(−0.07,0.07)$ ，将偏差参数清除为零。MXNet的init模块提供了多种预设的初始化方法。
### 2.2.1 使用内置初始化
+ 初始化为0.01高斯数据变量
+ force_reinit=True 即便初始化化过同样执行

In [11]:
net.initialize(init=init.Normal(sigma=0.01), force_reinit=True)
net[0].weight.data()[0]

array([0.01142411, 0.01085601, 0.00784212, 0.02014164])

+ 使用Constant初始化为常数

In [12]:
net.initialize(init=init.Constant(33), force_reinit=True)
net[0].weight.data()[0]

array([33., 33., 33., 33.])

+ 不同block执行同的初始化。

In [13]:
net[0].weight.initialize(init=init.Xavier(), force_reinit=True)
net[1].initialize(init=init.Constant(42), force_reinit=True)

## 2.2.2 自定义初始化
重新实现Initializer类中_init_weight带有张量参数（data）并为其分配所需的初始化值的函数即可。

In [14]:
class MyInit(init.Initializer):
    def _init_weight(self, name, data):
        print('Init', name, data.shape)
        data[:] = np.random.uniform(-10, 10, data.shape)
        data *= np.abs(data) >= 5

net.initialize(MyInit(), force_reinit=True)
net[0].weight.data()[:2]

Init dense10_weight (8, 4)
Init dense11_weight (1, 8)


array([[ 0.       , -5.4022765, -6.868823 , -0.       ],
       [-6.6433706,  0.       , -5.392142 ,  0.       ]])

+ 可以直接修改设置参数

In [15]:
net[0].weight.data()[:] += 1
net[0].weight.data()[0, 0] = 42
net[0].weight.data()[0]

array([42.       , -4.4022765, -5.868823 ,  1.       ])

## 2.3 捆绑参数
我们想跨多个层共享参数。让我们看看如何优雅地做到这一点。接下来，我们分配一个密集层，然后专门使用其参数来设置另一个层的参数。
+ 在层之间通过shared标志捆绑
+ 在修改了第一层的权重之后，一二层的权重仍然相同

In [16]:
net = nn.Sequential()

shared = nn.Dense(8, activation='relu')
net.add(nn.Dense(8, activation='relu'),
        shared,
        nn.Dense(8, activation='relu', params=shared.params),
        nn.Dense(10))
net.initialize()

X = np.random.uniform(size=(2, 20))
net(X)

# 看看参数是否相同
print(net[1].weight.data()[0] == net[2].weight.data()[0])
net[1].weight.data()[0, 0] = 100
# 修改一层的参数，确认参数是否捆绑成功
print(net[1].weight.data()[0] == net[2].weight.data()[0])

[ True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True]


# 3.延迟初始化
前使用Gluon创建的全连接层都没有指定输入个数。例如，在上一节使用的多层感知机net里，我们创建的隐藏层仅仅指定了输出大小为256。当调用initialize函数时，由于隐藏层输入个数依然未知，系统也无法得知该层权重参数的形状。只有在当我们将形状是(2, 20)的输入X传进网络做前向计算net(X)时，系统才推断出该层的权重参数形状为(256, 20)。因此，这时候我们才能真正开始初始化参数。

In [17]:
from mxnet import init, np, npx
from mxnet.gluon import nn
npx.set_np()

def get_net():
    net = nn.Sequential()
    net.add(nn.Dense(256, activation='relu'))
    net.add(nn.Dense(10))
    return net

net = get_net()

虽然存在参数对象，但每个图层的输入尺寸都列为-1。MXNet使用特殊值-1表示参数维仍然未知。

In [18]:
net.initialize()
net.collect_params()

sequential4_ (
  Parameter dense16_weight (shape=(256, -1), dtype=float32)
  Parameter dense16_bias (shape=(256,), dtype=float32)
  Parameter dense17_weight (shape=(10, -1), dtype=float32)
  Parameter dense17_bias (shape=(10,), dtype=float32)
)

+ 一旦我们知道输入维数为20，框架就可以通过插入值20来识别第一层的权重矩阵的形状。识别出第一层的形状后，框架将前进至第二层，依此类推计算图形，直到知道所有形状。

In [19]:
X = np.random.uniform(size=(2, 20))
net(X)

net.collect_params()

sequential4_ (
  Parameter dense16_weight (shape=(256, 20), dtype=float32)
  Parameter dense16_bias (shape=(256,), dtype=float32)
  Parameter dense17_weight (shape=(10, 256), dtype=float32)
  Parameter dense17_bias (shape=(10,), dtype=float32)
)

# 4. 自定义层
深度学习成功的一个因素是，可以以创造性的方式组成各种层来设计适合各种任务的架构。例如，研究人员发明了专门用于处理图像，文本，遍历顺序数据以及执行动态编程的图层。迟早，您将遇到或发明深度学习框架中尚不存在的层。在这些情况下，您必须构建一个自定义层。
## 4.1 无参图层
构建一个自定义图层，该图层没有自己的任何参数。只需要继承基层类并实现前向传播函数即可。

In [20]:
from mxnet import gluon, np, npx
from mxnet.gluon import nn
npx.set_np()

class CenteredLayer(nn.Block):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def forward(self, X):
        return X - X.mean()

## 4.2 有参图层
可以使用内置函数来创建参数，这些参数提供一些基本的内务管理功能。特别是，它们控制访问，初始化，共享，保存和加载模型参数。这样，除其他好处外，我们无需为每个自定义层编写自定义序列化例程。
+ 需要两个参数，一个代表重量，另一个代表偏差
+ 默认使用ReLU激活激活函数
+ 输入参数：in_units和units，分别表示输入和输出的数量

In [21]:
class MyDense(nn.Block):
    def __init__(self, units, in_units, **kwargs):
        super().__init__(**kwargs)
        self.weight = self.params.get('weight', shape=(in_units, units))
        self.bias = self.params.get('bias', shape=(units,))

    def forward(self, x):
        linear = np.dot(x, self.weight.data(ctx=x.ctx)) + self.bias.data(
            ctx=x.ctx)
        return npx.relu(linear)

# 5 模型参数保存
在进行长时间的培训时，最佳实践是定期保存中间结果（检查点），以确保在服务器电源线上跳闸时，我们不会损失几天的计算时间。
## 5.1 保存加载张量
对于单个张量，我们可以直接调用load和save 函数分别读取和写入它们。这两个函数都要求我们提供一个名称，并save要求将要保存的变量作为输入。

In [22]:
from mxnet import np, npx
from mxnet.gluon import nn
npx.set_np()

x = np.arange(4)
npx.save('x-file', x)

x2=npx.load('x-file')
x2

[array([0., 1., 2., 3.])]

## 5.2 保存加载模型
保存单个权重向量（或其他张量）很有用，但是如果我们要保存（并在以后加载）整个模型，这将非常繁琐。毕竟，我们可能遍布数百个参数组。因此，深度学习框架提供了内置功能来加载和保存整个网络
### 5.2.1 Block
模型参数的保存和加载， Block 只能保存网络参数

load参数：

+ allow_missing: True时表示：网络结构中存在, 参数文件中不存在参数，不加载
+ ignore_extra: True时表示: 参数文件中存在，网络结构中不存在的参数，不加载
+ cast_dtype: True时：将从检查点加载的NDArray的数据类型强制转换为Parameter提供的dtype
+ dtype_source: 必须为{‘current’，‘saved’}，仅当cast_dtype = True时有效，指定用于投射参数的dtype的来源


In [23]:
net.save_parameters("linear_regression.params")
net.load_parameters("linear_regression.params", ctx=npx.cpu(), allow_missing=False,
                    ignore_extra=False, cast_dtype=False, dtype_source='current')

### 5.2.2 HybridBlock
使用HybridBlock可以同时保存网络结构和参数，通过export导出，import进行加载

In [24]:
from mxnet.gluon import SymbolBlock

net = nn.HybridSequential()
net.add(nn.Dense(1))

net.initialize(init.Normal(sigma=0.02))
net.hybridize()
net(X)
net.export("net1", epoch=1)


net = SymbolBlock.imports(symbol_file='net1-symbol.json', input_names=['data'], param_file='net1-0001.params', 
                         ctx = npx.cpu())


/opt/conda/lib/python3.7/site-packages/mxnet/gluon/block.py:1512: UserWarning: Cannot decide type for the following arguments. Consider providing them as input:
	data: None
  input_sym_arg_type = in_param.infer_type()[0]
